In [1]:
from kaggle_secrets import UserSecretsClient

In [2]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [3]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git
    

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 70 (delta 25), reused 41 (delta 6), pack-reused 0
Unpacking objects: 100% (70/70), 27.02 MiB | 6.21 MiB/s, done.


In [ ]:
# %cd /kaggle/working/llama-3-finetune

In [ ]:
# !git checkout -b Update-Eval-Notebook

In [ ]:
# !git pull origin Update-Eval-Notebook

In [4]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [73]:
from datasets import load_dataset, load_metric
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch
import evaluate
import re

In [9]:
split_ratio = 0.2
seed = 42

In [10]:
dataset_name = "CohereForAI/aya_dataset"
dataset_type = "aya"

In [11]:
lang_multi = "Nepali"
_, aya_nepali_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

Generating train split:   0%|          | 0/202362 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4002 [00:00<?, ? examples/s]

In [15]:
aya_nepali_test = aya_nepali_test.select(range(200)) 
aya_nepali_test

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 200
})

In [16]:
lang_multi = "Hindi"
_, aya_hindi_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

In [17]:
aya_hindi_test = aya_hindi_test.select(range(200))
aya_hindi_test

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 200
})

In [18]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"
_, alpaca_nepali_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Generating train split:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

In [19]:
alpaca_nepali_test = alpaca_nepali_test.select(range(200))
alpaca_nepali_test

Dataset({
    features: ['instruction', 'input', 'output', 'id'],
    num_rows: 200
})

In [20]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"
_, alpaca_hindi_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [21]:
alpaca_hindi_test = alpaca_hindi_test.select(range(200))
alpaca_hindi_test

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 200
})

In [28]:
alpaca_prompt_hindi = """Below is an instruction in Hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in Hindi that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [29]:
alpaca_prompt_nepali = """Below is an instruction in Nepali that describes a task, paired with an input also in nepali that provides further context. Write a response in Nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [24]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [96]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def run_inference(dataset, prompt, is_alpaca_format=True):
    bleu_metric = evaluate.load("bleu")
    rouge_metric = evaluate.load("rouge")

    def update_row(row, is_alpaca_format=is_alpaca_format):
        if is_alpaca_format:
            instruction = row["instruction"]
            input_text = row["input"]
            prompt_text = prompt.format(instruction, input_text, "")
            target_text = row["output"]
        else:
            inputs = row["inputs"]
            prompt_text = prompt.format(inputs, "")
            target_text = row["targets"]

        inputs = tokenizer([prompt_text], return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
        generated_text = tokenizer.batch_decode(outputs)[0]
        response_pattern = r'### Response:\s*(.*?)\s*(?=### Explanation:)'
        match = re.search(response_pattern, generated_text, re.DOTALL)

        if match:
            response = match.group(1).strip()
        else:
            response = "Response not found in the model output."
            
        bleu_score = bleu_metric.compute(predictions=[response], references=[target_text])["bleu"]
        rouge_score = rouge_metric.compute(predictions=[response], references=[target_text])["rouge1"]
        
        return {
            "generated_text": generated_text,
            "model_response": response,
            "bleu_score": bleu_score,
            "rouge_score": rouge_score,
        }
    
    updated_dataset = dataset.map(update_row)
    return updated_dataset

In [30]:
aya_prompt_nepali = """Below is an input in Nepali that is a request or context. Write a response to the instruction or complete the context in given input in Nepali.

### Input:
{}

### Response:
{}"""

In [31]:
aya_prompt_hindi = """Below is an input in Hindi that is a request or context. Write a response to the instruction or complete the context in given input in Hindi.

### Input:
{}

### Response:
{}"""

In [99]:
alpaca_hindi_test= run_inference(alpaca_hindi_test, alpaca_prompt_hindi, is_alpaca_format=True)
alpaca_nepali_test = run_inference(alpaca_nepali_test, alpaca_prompt_nepali, is_alpaca_format=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [100]:
for i, row in enumerate(alpaca_nepali_test):
    rouge_score = row["rouge_score"]
    bleu_score = row["bleu_score"]
    print(rouge_score, bleu_score)

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0002015058091179328
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.15384615384615383 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.13333333333333333 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 1.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.

In [101]:
aya_hindi_test = run_inference(aya_hindi_test, aya_prompt_nepali, is_alpaca_format=False)
aya_nepali_test = run_inference(aya_nepali_test, aya_prompt_nepali, is_alpaca_format=False)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [106]:
aya_nepali_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/aya_nepali_test.csv")
aya_hindi_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/aya_hindi_test.csv")
alpaca_nepali_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/alpaca_nepali_test.csv")
alpaca_hindi_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/alpaca_hindi_test.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

483742

In [109]:
import numpy as np
avg_bleu_score = np.mean(alpaca_nepali_test["bleu_score"])
avg_rouge_score = np.mean(aya_nepali_test["rouge_score"])
print(avg_bleu_score, avg_rouge_score)

0.007797849987012127 0.0
